# Setup

In [58]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%autoreload 2

In [3]:
import torch

In [18]:
from src.vfm.connection import Connection
from src.vfm.preprocessor import Preprocessor
import pandas as pd
from src.vfm.model.nn.swt_tft import SWTTFTModel

In [66]:
connection = Connection()
# Get client
client = connection.get_client()

In [69]:
well = 'W06'
df_raw = connection.get_data(client, well = well, start=pd.Timestamp('2019-01-01'), end=pd.Timestamp('2023-01-31'))
df_raw.shape

(2085243, 13)

In [7]:
oldest_ts_per_column = df_raw.notna().apply(lambda col: col.index[col].min())
oldest_ts_per_column

whp            2019-01-30 18:22:00
wht            2019-01-30 18:22:00
dhp            2019-01-30 18:22:00
dht            2019-01-30 18:22:00
choke          2019-01-30 18:22:00
dcp            2019-01-30 18:22:00
qo_well_test   2019-01-30 20:39:00
qg_well_test   2019-01-30 20:39:00
qw_well_test   2019-01-30 20:39:00
qo_mpfm        2019-01-30 18:22:00
qg_mpfm        2019-01-30 18:22:00
wc_mpfm        2019-01-30 18:22:00
dtype: datetime64[ns]

In [8]:
latest_ts_per_column = df_raw.notna().iloc[::-1].idxmax()
latest_ts_per_column

whp            2023-01-17 11:19:00
wht            2023-01-17 11:19:00
dhp            2023-01-17 11:19:00
dht            2023-01-17 11:19:00
choke          2023-01-17 11:19:00
dcp            2023-01-17 11:19:00
qo_well_test   2023-01-17 20:24:00
qg_well_test   2023-01-17 20:24:00
qw_well_test   2023-01-17 20:24:00
qo_mpfm        2023-01-17 11:19:00
qg_mpfm        2023-01-17 11:19:00
wc_mpfm        2023-01-17 11:19:00
dtype: datetime64[ns]

In [ ]:
timestamps = df_raw.index[df_raw["qo_well_test"].notna()].to_list()
timestamps

[Timestamp('2019-01-30 20:39:00'),
 Timestamp('2019-01-31 04:57:00'),
 Timestamp('2019-01-31 13:30:00'),
 Timestamp('2019-03-02 15:21:00'),
 Timestamp('2019-03-04 08:21:00'),
 Timestamp('2019-03-10 17:42:00'),
 Timestamp('2019-04-14 16:54:00'),
 Timestamp('2019-04-15 16:55:00'),
 Timestamp('2019-04-15 23:44:00'),
 Timestamp('2019-04-16 07:08:00'),
 Timestamp('2019-05-22 17:07:00'),
 Timestamp('2019-05-24 05:19:00'),
 Timestamp('2019-05-24 07:50:00'),
 Timestamp('2019-05-24 10:48:00'),
 Timestamp('2019-05-24 17:02:00'),
 Timestamp('2019-05-25 20:16:00'),
 Timestamp('2019-05-26 05:12:00'),
 Timestamp('2019-05-26 06:59:00'),
 Timestamp('2019-05-26 10:20:00'),
 Timestamp('2019-05-26 18:57:00'),
 Timestamp('2019-05-27 01:57:00'),
 Timestamp('2019-05-27 08:01:00'),
 Timestamp('2019-07-05 09:49:00'),
 Timestamp('2019-07-05 22:05:00'),
 Timestamp('2019-08-27 21:52:00'),
 Timestamp('2019-08-28 01:12:00'),
 Timestamp('2019-08-28 06:02:00'),
 Timestamp('2019-08-28 08:06:00'),
 Timestamp('2019-09-

In [70]:
preprocessor = Preprocessor(df=df_raw)
df = preprocessor.preprocess_timeseries()
df.shape

(2085243, 14)

# Sparse Well-Test Temporal Fusion Transformer Model - Well 06

In [92]:
swt_tft_model = SWTTFTModel(dependent_variables=["qo_well_test", "qg_well_test", "qw_well_test"],
                            independent_variables=["dhp", "dht", "whp", "wht", "choke", "dcp"],
                             df=df)

c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


In [86]:
swt_tft_model.train()

c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params | Mode  | FLOPs
--------------------------------------------------------------------------------------------------------
0  | loss                               | MultiLoss                       | 0      | train | 0    
1  | logging_metrics                    | ModuleList                      | 0      | train | 0    
2  | input_embeddings                   | MultiEmbedding                  | 1      | train | 0    
3  | prescalers                         | ModuleDict                      | 480    | train | 0    
4  | static_variable_selection          | VariableSelectionNetwork        | 14.0 K | train | 0    
5  | encoder_variable_selection         | VariableSel

<class 'pytorch_forecasting.models.temporal_fusion_transformer._tft.TemporalFusionTransformer'>


c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=21` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 0/521204 [00:00<?, ?it/s]

IndexError: index 1 is out of bounds for dimension 1 with size 1

In [ ]:
import matplotlib.pyplot as plt

for well_idx in range(1):  # Plot all workload timeseries
    # for field_idx in range(len(target_fields)):
        fig, ax = plt.subplots(figsize=(10, 4))
        best_tft.plot_prediction(x=raw_predictions.x, out=raw_predictions.output, idx=well_idx, add_loss_to_title=False, ax=ax)
        current_title = ax.get_title()
        new_title = f' {well} qg_mpfm - known reals {time_varying_known_reals} \n' + current_title
        ax.set_title(new_title)